In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import variance_inflation_factor
from scipy.stats.mstats import winsorize

In [2]:
data_df = pd.read_csv('regression data.csv')
data_df = data_df.drop('station_geom', axis = 1)        # don't need geometry for regression

# Cleaning

In [3]:
# the WTC Cortlandt station was still being rebuilt and didn't reopen until 2018 → drop any rows with 0 values for ridership, population, employees
print(data_df[data_df['ridership_2013'] == 0])
data_df = data_df[(data_df['ridership_2013'] > 0) & (data_df['ridership_2018'] > 0)]
data_df = data_df[(data_df['population_2013'] > 0) & (data_df['population_2018'] > 0)]
data_df = data_df[(data_df['employee_count_2013'] > 0) & (data_df['employee_count_2018'] > 0)]

# keep relevent columns → the change columns, drop raw 2013 & 2018 counts
data_df = data_df.drop(['population_2013','population_2018','employee_count_2013','employee_count_2018','business_count_2013','business_count_2018','ridership_2013','ridership_2018'], axis = 1)

           station  population_2013  population_2018  employee_count_2013  \
301  WTC Cortlandt          79710.0          88865.0             274262.0   

     employee_count_2018  business_count_2013  business_count_2018    borough  \
301             308723.0              12976.0              12745.0  Manhattan   

     route_count  ridership_2013  ridership_2018  rider_change  pop_change  \
301            1               0            3558          3558      9155.0   

     emp_change  bus_change  
301     34461.0      -231.0  


### Skew & Winsorizing

In [4]:
data_df.drop(['station','route_count','borough'], axis = 1).skew()

        # ridership change and employee change is very skewed
        # business change is moderately skewed

        # need to winsorize

rider_change   -4.643054
pop_change     -0.289284
emp_change      2.919333
bus_change     -0.733022
dtype: float64

In [5]:
# save new df for winsorizing
win_df = data_df.drop(['station','route_count','borough'], axis = 1)

win_df['rider_change'] = winsorize(win_df['rider_change'], limits = [0.01, 0.01])
win_df['emp_change'] = winsorize(win_df['emp_change'], limits = [0.01, 0.01])

win_df.skew()

rider_change   -2.942078
pop_change     -0.289284
emp_change      2.727304
bus_change     -0.733022
dtype: float64

In [6]:
# still extremely skewed after 1% winsorizing → try 5%
win_df = data_df.drop(['station','route_count','borough'], axis = 1)

win_df['rider_change'] = winsorize(win_df['rider_change'], limits = [0.05, 0.05])
win_df['emp_change'] = winsorize(win_df['emp_change'], limits = [0.05, 0.05])

win_df.skew()

rider_change   -0.919157
pop_change     -0.289284
emp_change      1.173557
bus_change     -0.733022
dtype: float64

In [7]:
# still a bit high but more accepting → apply to our dataset
data_df['rider_change'] = winsorize(data_df['rider_change'], limits = [0.05, 0.05])
data_df['emp_change'] = winsorize(data_df['emp_change'], limits = [0.05, 0.05])

### Add interaction terms

In [8]:
data_df['pop_x_emp'] = data_df['pop_change'] * data_df['emp_change']            # does having growth in both population and employment amplify effect?
data_df['route_x_pop'] = data_df['route_count'] * data_df['emp_change']         # does population growth matter more for stations with more train lines?
data_df['route_x_emp'] = data_df['route_count'] * data_df['emp_change']         # does employment growth matter more for stations with more train lines?

In [9]:
X = data_df.drop(['station','route_count','borough'], axis = 1)

# add intercept
X = sm.add_constant(X)
vif_data = pd.DataFrame()
vif_data['feature'] = X.columns
vif_data['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
print(vif_data)

        feature       VIF
0         const  4.097272
1  rider_change  1.062860
2    pop_change  3.413969
3    emp_change  3.662688
4    bus_change  1.254202
5     pop_x_emp  4.833814
6   route_x_pop       inf
7   route_x_emp       inf


C:\Users\marcu\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\statsmodels\stats\outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


In [10]:
        # route_x_pop and route_x_emp   → perfect collinear         → drop these terms
        # pop_x_emp                     → high but under 5          → keep for now
        # others are low to moderate, drop the two route interactions and look again

X = data_df.drop(['station','route_count','borough','route_x_pop','route_x_emp'], axis = 1)

# add intercept
X = sm.add_constant(X)
vif_data = pd.DataFrame()
vif_data['feature'] = X.columns
vif_data['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
print(vif_data)

        feature       VIF
0         const  4.077675
1  rider_change  1.038346
2    pop_change  3.367031
3    emp_change  2.136912
4    bus_change  1.188147
5     pop_x_emp  4.808949


In [11]:
# pop_x_emp still high but under 5 (and it is an interaction so some collinearity is expected)
# apply drops to dataset
data_df= data_df.drop(['route_x_pop','route_x_emp'], axis = 1)

# Model

In [12]:
# make Manhattan the baseline borough as it's at the core of the transit system
data_df['borough'] = pd.Categorical(data_df['borough'], categories=['Manhattan', 'Brooklyn', 'Queens', 'Bronx'])

In [13]:
formula = """
rider_change ~ pop_change 
    + borough
    + emp_change
    + bus_change
    + route_count
    + pop_x_emp
"""

model = smf.ols(formula = formula, data = data_df).fit(cov_type = 'HC3')
model.summary()

C:\Users\marcu\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\statsmodels\base\model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 8, but rank is 7
  warnings.warn('covariance of constraints does not have full '


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           rider_change   R-squared:                       0.159
Model:                            OLS   Adj. R-squared:                  0.137
Method:                 Least Squares   F-statistic:                     6.835
Date:                Sun, 19 Oct 2025   Prob (F-statistic):           1.50e-07
Time:                        22:19:31   Log-Likelihood:                -2636.9
No. Observations:                 313   AIC:                             5292.
Df Residuals:                     304   BIC:                             5325.
Df Model:                           8                                         
Covariance Type:                  HC3                                         
=======================================================================================
                          coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept            -190.8274    266.088     -0.717      0.473    -712.350     330.696
borough[T.Brooklyn]   642.1156    282.106      2.276      0.023      89.198    1195.034
borough[T.Queens]     230.4907    285.175      0.808      0.419    -328.441     789.423
borough[T.Bronx]      156.4098    232.953      0.671      0.502    -300.170     612.990
pop_change              0.0215      0.014      1.498      0.134      -0.007       0.050
emp_change              0.0002      0.011      0.016      0.987      -0.022       0.022
bus_change             -0.1350      0.306     -0.441      0.659      -0.735       0.465
route_count          -324.7065     67.152     -4.835      0.000    -456.322    -193.091
pop_x_emp            -7.28e-08   9.17e-07     -0.079      0.937   -1.87e-06    1.72e-06
==============================================================================
Omnibus:                       15.157   Durbin-Watson:                   1.958
Prob(Omnibus):                  0.001   Jarque-Bera (JB):               17.085
Skew:                          -0.461   Prob(JB):                     0.000195
Kurtosis:                       3.678   Cond. No.                     1.11e+09
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC3)
[2] The condition number is large, 1.11e+09. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

#### Make summary more readable

In [14]:
# use * for significance
def significance_stars(p):
    if p < 0.01:
        return '***'
    elif p < 0.05:
        return '**'
    elif p < 0.10:
        return '*'
    else:
        return 'NS'

In [15]:
results = model

# create summary dataframe
summary_df = pd.DataFrame({
    'Variable': results.params.index,
    'Coefficient': results.params.values.round(1),
    'p_value': results.pvalues.values
})

# use * for significance
summary_df['Significance'] = summary_df['p_value'].apply(significance_stars)
summary_df = summary_df[['Variable', 'Coefficient', 'Significance']]

# rename variables for better readability
readable_names = {
    'Intercept':'Intercept (baseline when all variables = 0)',
    'borough[T.Brooklyn]':'Brooklyn',
    'borough[T.Queens]':'Queens',
    'borough[T.Bronx]':'Bronx',
    'pop_change':'Change in Population',
    'emp_change':'Change in Employee Count',
    'bus_change':'Change in Business Count',
    'route_count':'Number of Subway Lines',
    'pop_x_emp':'Population x Employees'
}

summary_df['Variable'] = summary_df['Variable'].replace(readable_names)

In [16]:
# insert rows for headers
boro_header = pd.DataFrame([['Borough Category — Relative to Manhattan','','']], columns = summary_df.columns)
summary_df = pd.concat([summary_df.iloc[:1], boro_header, summary_df.iloc[1:]]).reset_index(drop = True)

cont_header = pd.DataFrame([['Continuous Variables','','']], columns = summary_df.columns)
summary_df = pd.concat([summary_df.iloc[:5], cont_header, summary_df.iloc[5:]]).reset_index(drop = True)

interact_header = pd.DataFrame([['Interaction Terms','','']], columns = summary_df.columns)
summary_df = pd.concat([summary_df.iloc[:10], interact_header, summary_df.iloc[10:]]).reset_index(drop = True)

# insert spacing
blank_row = pd.DataFrame([['','','']], columns=summary_df.columns)
summary_df = pd.concat([summary_df.iloc[:1], blank_row, summary_df.iloc[1:]]).reset_index(drop = True)
summary_df = pd.concat([summary_df.iloc[:6], blank_row, summary_df.iloc[6:]]).reset_index(drop = True)
summary_df = pd.concat([summary_df.iloc[:12], blank_row, summary_df.iloc[12:]]).reset_index(drop = True)

In [17]:
print('OLS Regression Results')
print('')
print('r^2:', results.rsquared.round(4))
print('')
print('Legend:')
print('Significance: *** p<0.01 | ** p<0.05 | * p<0.10 | NS = Not Significant')
print('Coefficient: expected change in annual riders (in thousands)')
summary_df

OLS Regression Results

r^2: 0.1593

Legend:
Significance: *** p<0.01 | ** p<0.05 | * p<0.10 | NS = Not Significant
Coefficient: expected change in annual riders (in thousands)


,Variable,Coefficient,Significance
0,Intercept (baseline when all variables = 0),-190.8,NS
1,,,
2,Borough Category — Relative to Manhattan,,
3,Brooklyn,642.1,**
4,Queens,230.5,NS
5,Bronx,156.4,NS
6,,,
7,Continuous Variables,,
8,Change in Population,0.0,NS
9,Change in Employee Count,0.0,NS
